In [11]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import os 
import sys
from torch.utils.data import Dataset, DataLoader
import pandas as pd

In [ ]:
#MAYBE INCLUDE day of the week, month, day of the year as input aswell

In [29]:
#TODO make supervised learning dataset from csv files
#TODO build dataloader
solar_data = np.genfromtxt('data\\training_data\\train_solar.csv', delimiter=',', skip_header=1)
wind_data = np.genfromtxt('data\\training_data\\train_wind.csv', delimiter=',',skip_header=1)

In [37]:
solar_data.shape
test_data = np.random.rand(6, 6)


(17544, 6)

In [36]:
N = solar_data.shape[0]
l = 5
n = 1
m = 1
k = N - (l + m + n)



In [49]:
def move_sliding_window(data, window_size, inputs_cols_indices, label_col_index):
    """
    data: numpy array including data
    window_size: size of window
    inputs_cols_indices: col indices to include
    """

    # (# instances created by movement, seq_len (timestamps), # features (input_len))
    inputs = np.zeros((len(data) - window_size, window_size, len(inputs_cols_indices)))
    labels = np.zeros(len(data) - window_size)

    for i in range(window_size, len(data)):
        inputs[i - window_size] = data[i - window_size : i, inputs_cols_indices]
        labels[i - window_size] = data[i, label_col_index]
    inputs = inputs.reshape(-1, window_size, len(inputs_cols_indices))
    labels = labels.reshape(-1, 1)
    print(inputs.shape, labels.shape)

    return inputs, labels

In [48]:
print(test_data)
inputs_test = np.array([test_data[i:i+l] for i in range(k)])
outputs_test = np.array([test_data[i+l:i+l+m,0] for i in range(k)])
print(inputs_test[0])
print(outputs_test[0])

[[0.93305667 0.26788469 0.43388759 0.25778974 0.55415499 0.69550187]
 [0.57719415 0.25277559 0.78328127 0.17285781 0.56988906 0.93663889]
 [0.91785678 0.50335685 0.01618584 0.36799884 0.88730611 0.47364465]
 [0.23141142 0.08473108 0.30973464 0.95463363 0.7272642  0.01182994]
 [0.6238014  0.55598621 0.43403017 0.39748669 0.61125417 0.58531857]
 [0.72361274 0.51563549 0.89944318 0.95362713 0.15512222 0.60894126]
 [0.9359016  0.95937447 0.47482583 0.91159402 0.09077589 0.80830229]]
[[0.93305667 0.26788469 0.43388759 0.25778974 0.55415499 0.69550187]
 [0.57719415 0.25277559 0.78328127 0.17285781 0.56988906 0.93663889]
 [0.91785678 0.50335685 0.01618584 0.36799884 0.88730611 0.47364465]
 [0.23141142 0.08473108 0.30973464 0.95463363 0.7272642  0.01182994]
 [0.6238014  0.55598621 0.43403017 0.39748669 0.61125417 0.58531857]]
[0.72361274]


C:\Users\belau\AppData\Local\Temp\ipykernel_1296\1056773489.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  inputs_test = np.array([test_data[i:i+l] for i in range(k)])
C:\Users\belau\AppData\Local\Temp\ipykernel_1296\1056773489.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  outputs_test = np.array([test_data[i+l:i+l+m,0] for i in range(k)])


In [52]:
i, l = move_sliding_window(test_data, 5, range(6), 0)
print(i[0])
print(l[0])

(2, 5, 6) (2, 1)
[[0.93305667 0.26788469 0.43388759 0.25778974 0.55415499 0.69550187]
 [0.57719415 0.25277559 0.78328127 0.17285781 0.56988906 0.93663889]
 [0.91785678 0.50335685 0.01618584 0.36799884 0.88730611 0.47364465]
 [0.23141142 0.08473108 0.30973464 0.95463363 0.7272642  0.01182994]
 [0.6238014  0.55598621 0.43403017 0.39748669 0.61125417 0.58531857]]
[0.72361274]


In [45]:
inputs = torch.tensor([solar_data[i:i+l] for i in range(k)], dtype=torch.float32)
#only need first column of data as output
output = torch.tensor([solar_data[i+l+m:i+l+m+n,0] for i in range(k)],dtype=torch.float32)
print(inputs.shape)
print(output.shape)


torch.Size([17537, 5, 6])
torch.Size([17537, 1])


In [46]:
#create dataset
class SolarDataset(Dataset):
    def __init__(self, inputs, output):
        self.inputs = inputs
        self.output = output

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return self.inputs[idx], self.output[idx]

solar_dataset = SolarDataset(inputs, output)
dataloader = DataLoader(solar_dataset, batch_size=32, shuffle=False, drop_last=True)


In [47]:
#save dataloader
torch.save(dataloader, 'data/solar_dataloader.pth')